In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp '/content/drive/MyDrive/sound_sentimen/train (1).csv' '/content/train.csv'
!cp '/content/drive/MyDrive/sound_sentimen/train (2).zip' '/content/train.zip'
!cp '/content/drive/MyDrive/sound_sentimen/val (2).zip' '/content/val.zip'
!cp '/content/drive/MyDrive/sound_sentimen/val.csv' '/content/val.csv'

In [3]:
!unzip '/content/train.zip'
!unzip '/content/val.zip'

Streaming output truncated to the last 5000 lines.
  inflating: train/73.wav            
  inflating: train/730.wav           
  inflating: train/7300.wav          
  inflating: train/7301.wav          
  inflating: train/7302.wav          
  inflating: train/7303.wav          
  inflating: train/7304.wav          
  inflating: train/7305.wav          
  inflating: train/7306.wav          
  inflating: train/7307.wav          
  inflating: train/7308.wav          
  inflating: train/7309.wav          
  inflating: train/731.wav           
  inflating: train/7310.wav          
  inflating: train/7311.wav          
  inflating: train/7312.wav          
  inflating: train/7313.wav          
  inflating: train/7314.wav          
  inflating: train/7315.wav          
  inflating: train/7316.wav          
  inflating: train/7317.wav          
  inflating: train/7318.wav          
  inflating: train/7319.wav          
  inflating: train/732.wav           
  inflating: train/7320.wav          

In [4]:
import librosa
import soundfile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [5]:
def extract_feature(file_name):
    
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        result = np.array([])
        mfccs = np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=40).T,axis=0)
        result = np.hstack((result,mfccs))
        stft = np.abs(librosa.stft(X))
        chroma = np.mean(librosa.feature.chroma_stft(S=stft,sr=sample_rate).T,axis=0)
        result = np.hstack((result,chroma))
        mel = np.mean(librosa.feature.melspectrogram(X,sr=sample_rate).T,axis=0)
        result = np.hstack((result,mel))
    return result
    #180 dimensional row (40+12+128)


In [6]:
def load_data(file_path,label_path,test_bool=False):
    x,y=[],[]
    if test_bool==False:
      data = pd.read_csv(os.path.join(label_path))
    for file in tqdm(os.listdir(file_path)):
        file_name = os.path.join(file_path + file)
        label_name = file.split('.')[0]
        features = extract_feature(file_name)
        x.append(features)
        if test_bool==False:
          label = data.loc[data['wav_id']==int(label_name),'label']
          y.append(label)
    
    if test_bool==False:
      return np.array(x),np.array(y)
    else:
      return np.array(x) 

In [7]:
train_path = '/content/train/'
train_file = '/content/train.csv'
X_train,Y_train = load_data(train_path,train_file)
print(X_train.shape)
print(Y_train.shape)

100%|██████████| 15000/15000 [08:12<00:00, 30.46it/s]


(15000, 180)
(15000, 1)


In [8]:
val_path = '/content/val/'
val_file = '/content/val.csv'
X_val,Y_val = load_data(val_path,val_file)
print(X_val.shape)
print(Y_val.shape)

100%|██████████| 2000/2000 [01:08<00:00, 29.38it/s]


(2000, 180)
(2000, 1)


In [20]:
from sklearn.ensemble import RandomForestClassifier
weights = {0:4.288, 1:4.263, 2:0.395}
model = RandomForestClassifier(n_estimators=1000, class_weight=weights)

In [21]:
del history
history = model.fit(X_train,Y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [22]:
print(history)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight={0: 4.288, 1: 4.263, 2: 0.395},
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [27]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [28]:
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
# model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [29]:
Y_val_pred=model.predict(X_val)
#Y_val_pre_arg = np.argmax(Y_val_pred,axis=1)  Remove for dl
print(Y_val_pred[0:5])
#print(Y_val_pre_arg[0:5])

[2 2 2 2 2]


In [30]:
#accuracy=accuracy_score(y_true=Y_val, y_pred=Y_val_pre_arg) Remove for DL

accuracy=accuracy_score(y_true=Y_val, y_pred=Y_val_pred) 

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 80.65%


In [31]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
def evaluate_model(X, y, model):
	# define evaluation procedure
	cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
	# evaluate model
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
	return scores

In [ ]:

scores = evaluate_model(X_val, Y_val, model)
# summarize performance


In [33]:
from numpy import mean
from numpy import std
print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean Accuracy: 0.807 (0.001)


In [25]:
!cp '/content/drive/MyDrive/sound_sentimen/test (1).zip' '/content/test.zip'
!unzip '/content/test.zip'

Streaming output truncated to the last 5000 lines.
  inflating: test/0.wav              
  inflating: test/1.wav              
  inflating: test/10.wav             
  inflating: test/100.wav            
  inflating: test/1000.wav           
  inflating: test/1001.wav           
  inflating: test/1002.wav           
  inflating: test/1003.wav           
  inflating: test/1004.wav           
  inflating: test/1005.wav           
  inflating: test/1006.wav           
  inflating: test/1007.wav           
  inflating: test/1008.wav           
  inflating: test/1009.wav           
  inflating: test/101.wav            
  inflating: test/1010.wav           
  inflating: test/1011.wav           
  inflating: test/1012.wav           
  inflating: test/1013.wav           
  inflating: test/1014.wav           
  inflating: test/1015.wav           
  inflating: test/1016.wav           
  inflating: test/1017.wav           
  inflating: test/1018.wav           
  inflating: test/1019.wav           

In [26]:
test_path = '/content/test/'
X_test = load_data(test_path,None,True)
print(X_test.shape)

100%|██████████| 5000/5000 [02:43<00:00, 30.53it/s]

(5000, 180)


In [34]:
df = pd.DataFrame(columns=['wav_id', 'label'])

In [36]:
Y_test_pred=model.predict(X_test)
#Y_test_pre_arg = np.argmax(Y_test_pred,axis=1) Remove for DL
print(Y_test_pred[0:5])
#print(Y_test_pre_arg.shape)

[2 2 2 2 2]


In [37]:
for i,y in tqdm(enumerate(Y_test_pred)):
  df = df.append({'wav_id':i,'label':y},ignore_index=True)
print(df.head())

5000it [00:07, 653.81it/s]

  wav_id label
0      0     2
1      1     2
2      2     2
3      3     2
4      4     2


In [38]:
df.to_csv("submission.csv", index=False)
try:
    from google.colab import files
    files.download('submission.csv') 
except:
    print("Option Only avilable in Google Colab")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [39]:
try:
    from google.colab import files
    files.download('/content/finalized_model.sav') 
except:
    print("Option Only avilable in Google Colab")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>